# Payload

In [1]:
PAYLOAD = {
    "datasetName": "random_10",
    "gaParameters": {
        "maxNumberOfIter":300,
        "populationSize": 100,
        "maximumWildCardCount": 1,
        "orientationWeights": [
            1,
            1,
            0.5
        ],
        "geneticAlgorithm": "simpleGa",
        "mate": "normalizedProbabilityVectorSum",
        "mutate": "flipOnePartAtRandom",
        "select": "tournament",
        "objective": "simple",
        "evaluator": "ga",
        "placingHeuristics": "corner",
        "populationDivisionCounts": {
            "elite": 0.2,
            "average": 0.6,
            "worst": 0.2,
            "children": 0.3,
            "mutant": 0.2,
            "winner": 0.2,
            "random": 0.1
        },
        "initialPopulationDivisionCounts": {
            "random": 0.7,
            "greedy": 0.3
        }
    },
    "objectiveParameters": {
        "name": "simple",
        "params": {
            "overlappingPenalizationConstant": 0,
            "outsideOfAllocatedAreaPenalizationConstant": 0
        }
    }
}

URL = "http://localhost:9000/compute/dataset"

HEADERS = {
  'Content-Type': 'application/json'
}

# Utils

In [2]:
import os
import time
import requests
import json
from pathlib import Path
import os
import shutil
from datetime import datetime
import math
import copy

def createDirectory(directory, removeAll=False):
    '''
    Creates a directory if it doesn't exist and removes all files
    from the directory if it already exists and removeAll is True.
    '''
    if os.path.exists(directory) and removeAll:
        shutil.rmtree(directory) # Remove all files in the directory
        print(f'All files in {directory} have been removed.')

    if not os.path.exists(directory):
        os.makedirs(directory) # Create the directory
        
    return directory

def checkforFileBlocking(outDir):
    start_time = time.time()  # Get the current time in seconds
    while True:
        if os.path.exists(os.path.join(outDir, "computation_result.json")):
            elapsed_time = time.time() - start_time  # Calculate the elapsed time in seconds
            print(f"{elapsed_time:.2f} sec")
            time.sleep(5)  # weit for file to be written and closed
            break
        else:
            time.sleep(1)  # Pause for 1 second before checking again
            
def findDataset(name):
    for root, dirs, files in os.walk('/home/slarty/Documents/master-thesis/src/impl/public/datasets'):
        for file in files:
            if file == f'{name}.json' or file == name:
                return os.path.join(root, file)
    return None

def calculateLayoutDiagonalLen(datasetName):
    with open(findDataset(datasetName), 'r') as file:
        layout = json.load(file)['data']['layout']
    return math.sqrt(layout['width']**2 + layout['height']**2)
    

# def setOverlappingConstant(data, dataset):
#     with open(findDataset(dataset), 'r') as file:
#         layout = json.load(file)['data']['layout']
#     data['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
    
def savePayload(payload, directory):
    os.makedirs(directory, exist_ok=True)
    with open(f'{directory}/payload.json', 'w') as file:
        json.dump(payload, file, indent=4)
    
def sendPayloadBlocking(payload, outDir):
    r = requests.request("POST", URL, headers=HEADERS, data=json.dumps(payload))
    print(r.json())
    src = r.json()['outputDirectory'] 
    out = outDir + '/' + src.split('/')[-1]
    savePayload(payload, out)
    checkforFileBlocking(src)
    shutil.copytree(src, out,dirs_exist_ok=True)
    
def createOutDir(name):
    directory = f'out/{name}_' +  datetime.now().strftime('%Y_%m_%d_%H:%M:%S')
    createDirectory(directory)
    return directory

def printRun(run, runMax):
    print(f'====== RUN {run}/{runMax} ======')
    
def setCommonAttributes(payload, dataset,size):
    print(f'{dataset}')
    payload['datasetName'] = dataset
    payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateLayoutDiagonalLen(dataset)
    payload['objectiveParameters']['params']['outsideOfAllocatedAreaPenalizationConstant'] = calculateLayoutDiagonalLen(dataset)
    payload['gaParameters']['populationSize'] = size * 50
    payload['gaParameters']['maxNumberOfIter'] = 300

# Hyperparameters

## 1. Max number of iter

In [11]:
# zopakovat nekolikrat pro maxNumberOfIter=500, mozna by slo i vic, treba 800 nebo 1000
outDirBase = createOutDir('max_number_of_iter')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 4

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('random_20',20)]:
        setCommonAttributes(payload, dataset, size)
        for maxNumberOfIter in [1000]:
            print(f'\tmaxNumberOfIter={maxNumberOfIter}')
            payload['gaParameters']['maxNumberOfIter'] = maxNumberOfIter
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

====== RUN 1/4 ======
random_10
	maxNumberOfIter=1000
{'id': 'random_10_F7ACC', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/135_random_10_F7ACC'}
685.68 sec
random_20
	maxNumberOfIter=1000
{'id': 'random_20_F68B3', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/136_random_20_F68B3'}
3422.42 sec
====== RUN 2/4 ======
random_10
	maxNumberOfIter=1000
{'id': 'random_10_EEE7D', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/137_random_10_EEE7D'}
676.67 sec
random_20
	maxNumberOfIter=1000
{'id': 'random_20_B95C2', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/138_random_20_B95C2'}
3396.38 sec
====== RUN 3/4 ======
random_10
	maxNumberOfIter=1000
{'id': 'random_10_BD44C', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/139_random_10_BD44C'}
681.68 sec
random_20
	m

## 2. Population size

In [12]:
# rerun for 10, 50, 100
outDirBase = createOutDir('population_size')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 5

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('random_20',20)]:
        setCommonAttributes(payload, dataset, size)
        for scalingFactor in [10, 50, 100]:
            populationSize = size * scalingFactor
            print(f'\tscalingFactor={scalingFactor}, populationSize={populationSize}')
            payload['gaParameters']['populationSize'] = populationSize
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

====== RUN 1/5 ======
random_10
	scalingFactor=10, populationSize=100
{'id': 'random_10_5C323', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/143_random_10_5C323'}
43.04 sec
	scalingFactor=50, populationSize=500
{'id': 'random_10_B72EF', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/144_random_10_B72EF'}
204.20 sec
	scalingFactor=100, populationSize=1000
{'id': 'random_10_75A1A', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/145_random_10_75A1A'}
413.41 sec
random_20
	scalingFactor=10, populationSize=200
{'id': 'random_20_D60D8', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/146_random_20_D60D8'}
204.20 sec
	scalingFactor=50, populationSize=1000
{'id': 'random_20_97C64', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/147_random_20_97C64'}
1016.01 sec
	scal

## 3. Maximum wild card count

In [7]:
# RUN ALWAYS LAST!!!
outDirBase = createOutDir('maximum_wild_card_count')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 4

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10)]:
        setCommonAttributes(payload, dataset, size)
        for maximumWildCardCount in range(5,10):
            print(f'\tmaximumWildCardCount={maximumWildCardCount}')
            payload['gaParameters']['maximumWildCardCount'] = maximumWildCardCount
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

====== RUN 1/4 ======
random_10
	maximumWildCardCount=5
{'id': 'random_10_A2474', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/385_random_10_A2474'}
1333.35 sec
	maximumWildCardCount=6
{'id': 'random_10_819AA', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/386_random_10_819AA'}
1657.71 sec
	maximumWildCardCount=7
{'id': 'random_10_093E4', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/387_random_10_093E4'}
2242.28 sec
	maximumWildCardCount=8
{'id': 'random_10_48F00', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/388_random_10_48F00'}
2232.34 sec
	maximumWildCardCount=9
{'id': 'random_10_F0A84', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/389_random_10_F0A84'}
2323.33 sec
====== RUN 2/4 ======
random_10
	maximumWildCardCount=5
{'id': 'random_10_3242F', '

KeyboardInterrupt: 

## 4. Orientation weights

In [ ]:
outDirBase = createOutDir('orientation_weights')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 4

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('random_20', 20)]:
        setCommonAttributes(payload, dataset, size)
        for w in [0, 0.2, 0.5, 0.7, 1]:
            orientationWeights = [1,1,w]
            print(f'\torientationWeights={orientationWeights}')
            payload['gaParameters']['orientationWeights'] = orientationWeights
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

====== RUN 1/4 ======
random_10
	orientationWeights=[1, 1, 0]
{'id': 'random_10_D0426', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/173_random_10_D0426'}
245.26 sec
	orientationWeights=[1, 1, 0.2]
{'id': 'random_10_92E82', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/174_random_10_92E82'}
232.23 sec
	orientationWeights=[1, 1, 0.5]
{'id': 'random_10_64C46', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/175_random_10_64C46'}
203.20 sec
	orientationWeights=[1, 1, 0.7]
{'id': 'random_10_1F3BA', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/176_random_10_1F3BA'}
203.20 sec
	orientationWeights=[1, 1, 1]
{'id': 'random_10_2B93B', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/177_random_10_2B93B'}
206.21 sec
random_20
	orientationWeights=[1, 1, 0]
{'id': 'rand

## 5. Population division counts

In [ ]:
counts =  [
    # no random, add to children
    {
            "elite": 0.2,
            "average": 0.6,
            "worst": 0.2,
            "children": 0.4,
            "mutant": 0.2,
            "winner": 0.2,
            "random": 0
        },
    
        # "populationDivisionCounts": {
        #     "elite": 0.2,
        #     "average": 0.6,
        #     "worst": 0.2,
        #     "children": 0.3,
        #     "mutant": 0.2,
        #     "winner": 0.2,
        #     "random": 0.1
        # },
    # no elite, add to children and equaly to average|worst
    {
            "elite": 0.01,
            "average": 0.69,
            "worst": 0.3,
            "children": 0.49,
            "mutant": 0.2,
            "winner": 0.2,
            "random": 0.1
        },
    # no elite and random, add to children, add equaly to average|worst
    {
            "elite": 0.01,
            "average": 0.69,
            "worst": 0.3,
            "children": 0.59,
            "mutant": 0.2,
            "winner": 0.2,
            "random": 0
        }
]

outDirBase = createOutDir('population_division_counts')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 4

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('random_20', 20)]:
        setCommonAttributes(payload, dataset, size)
        for populationDivisionCounts in counts:
            print(f'\tpopulationDivisionCounts={populationDivisionCounts}')
            payload['gaParameters']['populationDivisionCounts'] = populationDivisionCounts
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

## 6. Initial population division counts

In [ ]:
outDirBase = createOutDir('initial_population_division_counts')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 4

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('random_20', 20)]:
        setCommonAttributes(payload, dataset, size)
        for randomPercent in [0, 0.3, 0.5, 0.8, 1]:
            print(f'\trandomPercent={randomPercent}')
            payload['gaParameters']['initialPopulationDivisionCounts']['random'] = randomPercent
            payload['gaParameters']['initialPopulationDivisionCounts']['greedy'] = 1 - randomPercent
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

## 7. Overlapping penalization constant

In [3]:
outDirBase = createOutDir('overlapping_penalization_constant')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 4

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('random_20', 20)]:
        setCommonAttributes(payload, dataset, size)
        for scalingFactor in [0.5, 1, 2, 3, 4]:
            overlappingPenalizationConstant = calculateLayoutDiagonalLen(dataset) * scalingFactor
            print(f'\tscalingFactor={scalingFactor}, overlappingPenalizationConstant={overlappingPenalizationConstant:.2f}')
            payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = overlappingPenalizationConstant
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

====== RUN 1/1 ======
random_20
	scalingFactor=3, overlappingPenalizationConstant=119.47
{'id': 'random_20_3CA55', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/316_random_20_3CA55'}
1043.08 sec
	scalingFactor=4, overlappingPenalizationConstant=159.30
{'id': 'random_20_585CA', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/317_random_20_585CA'}
1118.15 sec

Computation done


## 8. Outside of allocated area penalization constant

In [ ]:
# TODO1: run this again for zero
# TODO2: run this for 10 and 50 (or some other big numbers)
outDirBase = createOutDir('outside_of_allocated_area_penalization_constant')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 5

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('random_20', 20)]:
        setCommonAttributes(payload, dataset, size)
        for scalingFactor in [0,10,50]:
            outsideOfAllocatedAreaPenalizationConstant = calculateLayoutDiagonalLen(dataset) * scalingFactor
            print(f'\tscalingFactor={scalingFactor}, overlappingPenalizationConstant={outsideOfAllocatedAreaPenalizationConstant:.2f}')
            payload['objectiveParameters']['params']['outsideOfAllocatedAreaPenalizationConstant'] = outsideOfAllocatedAreaPenalizationConstant
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

====== RUN 1/5 ======
random_10
	scalingFactor=0, overlappingPenalizationConstant=0.00
{'id': 'random_10_25D0C', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/395_random_10_25D0C'}
182.18 sec
	scalingFactor=10, overlappingPenalizationConstant=306.10
{'id': 'random_10_ED053', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/396_random_10_ED053'}
181.18 sec
	scalingFactor=50, overlappingPenalizationConstant=1530.52
{'id': 'random_10_F1675', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/397_random_10_F1675'}
182.18 sec
random_20
	scalingFactor=0, overlappingPenalizationConstant=0.00
{'id': 'random_20_E2A5C', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/398_random_20_E2A5C'}
946.96 sec
	scalingFactor=10, overlappingPenalizationConstant=398.25
{'id': 'random_20_97EE8', 'outputDirectory': '/home/slarty/Docume

# Measurements

## Population size x best solution

In [15]:
outDirBase = createOutDir('population_size')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 5

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('cluster_3_10', 30), ('cluster_4_20',80)]:
        setOverlappingConstant(payload, dataset)
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 50
        payload['gaParameters']['maxNumberOfIter'] = 30
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/5 ======
{'id': 'cluster_3_10_3E78A', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/008_cluster_3_10_3E78A'}


KeyboardInterrupt: 

## Overlapping Penalization Constant

In [11]:
outDirBase = createOutDir('overlapping_penalization_constant')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 5

def getLayoutWidthAndHeight(dataset):
    with open(findDataset(dataset), 'r') as file:
        layout = json.load(file)['data']['layout']
    return layout['width'], layout['height']

def calculateDiagonalLength(dataset):
    w,h = getLayoutWidthAndHeight(dataset)
    return math.sqrt(w**2 + h**2)


def setOverlappingConstant(data, dataset):
    with open(findDataset(dataset), 'r') as file:
        layout = json.load(file)['data']['layout']
    data['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
    
for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('packing_10', 10)]:
        for k in [2, 3, 4, 6]:
            print(f'{dataset}, {k}')
            payload['datasetName'] = dataset
            payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateDiagonalLength(dataset) * k
            payload['gaParameters']['populationSize'] = size * 50
            payload['gaParameters']['maxNumberOfIter'] = 300
            payload['gaParameters']['maximumWildCardCount'] = 0
            sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/5 ======
random_10, 2
{'id': 'random_10_D9842', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/094_random_10_D9842'}
113.12 sec
random_10, 3
{'id': 'random_10_73195', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/095_random_10_73195'}
114.11 sec
random_10, 4
{'id': 'random_10_AD7EE', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/096_random_10_AD7EE'}
113.11 sec
random_10, 6
{'id': 'random_10_68D8F', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/097_random_10_68D8F'}
113.11 sec
packing_10, 2
{'id': 'packing_10_1457A', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/098_packing_10_1457A'}
94.09 sec
packing_10, 3
{'id': 'packing_10_B0ACF', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/099_packing_10_B0

## Random

In [20]:
outDirBase = createOutDir('random')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_20',20)]:
        setOverlappingConstant(payload, dataset)
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 50
        payload['gaParameters']['maxNumberOfIter'] = 300
        payload['gaParameters']['maximumWildCardCount'] = 1
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateDiagonalLength(dataset) * 8
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/1 ======
{'id': 'random_20_FDC76', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/148_random_20_FDC76'}
990.00 sec

Computation done


## Cluster

In [22]:
outDirBase = createOutDir('cluster')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

# 10.4 - testing maximum wild card count = 1
for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    # for dataset, size in [('cluster_3_6', 18)]:
    for dataset, size in [('cluster_4_5', 20)]:
        print(f'{dataset}')
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 50
        payload['gaParameters']['maxNumberOfIter'] = 300
        payload['gaParameters']['maximumWildCardCount'] = 1
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateDiagonalLength(dataset) * 8
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/1 ======
cluster_4_5
{'id': 'cluster_4_5_606A8', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/150_cluster_4_5_606A8'}
855.86 sec

Computation done


## Packing

In [ ]:
outDirBase = createOutDir('packing')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 5

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('packing_10',10), ('packing_20',20)]:
        setOverlappingConstant(payload, dataset)
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 100
        payload['gaParameters']['maxNumberOfIter'] = 1000
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

## Biased sparse cluster

In [25]:
outDirBase = createOutDir('biased_sparse_cluster')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('biased_sparse_cluster_3_5', 15)]:
        print(f'maximumWildCardCount={maximumWildCardCount}, dataset={dataset}')
        payload['datasetName'] = dataset
        payload['gaParameters']['maxNumberOfIter'] = 300
        payload['gaParameters']['populationSize'] = size * 50
        payload['gaParameters']['maximumWildCardCount'] = 1
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateDiagonalLength(dataset) * 8
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/1 ======
maximumWildCardCount=2, dataset=biased_sparse_cluster_3_5
{'id': 'biased_sparse_cluster_3_5_F769D', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/152_biased_sparse_cluster_3_5_F769D'}
686.71 sec

Computation done


## Maximum wild card count vs computation time

In [13]:
outDirBase = createOutDir('wild_card_comp_time')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 5

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('packing_10',10)]:
        setOverlappingConstant(payload, dataset)
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 100
        payload['gaParameters']['maxNumberOfIter'] = 300
        
        for maximumWildCardCount in range(0,size+1):
            print(maximumWildCardCount)
            payload['gaParameters']['maximumWildCardCount'] = maximumWildCardCount
            sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/5 ======
0
{'id': 'packing_10_9D560', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/139_packing_10_9D560'}
180.18 sec
1
{'id': 'packing_10_4B0EB', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/140_packing_10_4B0EB'}
309.31 sec
2
{'id': 'packing_10_94F18', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/141_packing_10_94F18'}


KeyboardInterrupt: 

## Bruteforce

In [12]:
outDirBase = createOutDir('brute')
payload = copy.deepcopy(PAYLOAD)

data = [('random_10',10), ('random_20',20), ('cluster_3_10',30)]
data = [('cluster_3_6', 18)]

for dataset, size in data:
    print(f'{dataset}')
    setOverlappingConstant(payload, dataset)
    payload['datasetName'] = dataset
    payload['gaParameters']['populationSize'] = 100 * size
    payload['gaParameters']['maxNumberOfIter'] = 1000
    payload['gaParameters']['geneticAlgorithm'] = 'probBrute'
    sendPayloadBlocking(payload, outDirBase)

print('\nComputation done')

random_10
775.77 sec
random_20
3908.93 sec
cluster_3_10
9184.24 sec

Computation done


## London National Gallery

In [32]:
outDirBase = createOutDir('london_nationa_gallery')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('london_gallery_wall',7)]:
        setOverlappingConstant(payload, dataset)
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 100
        payload['gaParameters']['maxNumberOfIter'] = 10
        payload['gaParameters']['maximumWildCardCount'] = 1
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateDiagonalLength(dataset) * 5
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/1 ======
{'id': 'london_gallery_wall_4C686', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/157_london_gallery_wall_4C686'}
6.01 sec

Computation done
